<a href="https://colab.research.google.com/github/hermanholmoy/TDT4109/blob/master/setfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!pip install setfit
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss
from sklearn.metrics import precision_recall_fscore_support


from setfit import SetFitModel, SetFitTrainer, sample_dataset
import pandas as pd
from datasets import Dataset
import numpy as np
import datetime



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
df = pd.read_excel(f"training_data_one_hot_060823.xlsx")


In [3]:
df = df.rename(columns={"chunked_text": "text"})
print(f"Found {len(df)} rows in dataset")

label_cols = df.columns[1:]
print(f"Using {len(label_cols)} labels")

df['one_hot_labels'] = list(df[label_cols].values)
df.head()

Found 56499 rows in dataset
Using 28 labels


,text,Agriculture,Air transportation,Battery,Bioenergy,CCUS,Coal,Combined heat and power (CHP),Defense,Electric grid,...,Pipeline,Rail transportation,Road transportation,Shipping,Solar,Storage,Waste,Water,Wind,one_hot_labels
0,"""ACCIONA Energía is today a financially strong...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"""In addition to the growing social awareness a...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,The Chairman of ACCIONA Energía reviewed the s...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,to be geo-strategically transcendental solutio...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"In this context, José Manuel Entrecanales assu...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
# Make dataset
# Assign index to each row
df['idx'] = range(len(df))

slim_df = df[["idx", "text", "one_hot_labels"]]
print(len(slim_df.index))
slim_df = slim_df.dropna()
print(len(slim_df.index))

lists = []
for row in slim_df["one_hot_labels"]:
  arr = list(row)
  lists.append(arr)


# Create Hugging Face dataset
slim_df["one_hot_labels"] = lists

dataset = Dataset.from_pandas(slim_df[:1000])


56499
56499


In [72]:
# Make balanced dataset
# tdf = slim_df.explode('one_hot_labels')
tdf = df
sums = df[label_cols].sum(axis=1)
# single_label = df[(sums == 1) | (sums == 0)]

num_samples_per_group = 50

# Sample rows from each group
dfs = []
for group in label_cols:
  balanced_df = pd.concat([group.sample(num_samples_per_group, replace=len(group) < num_samples_per_group) for _, group in tdf.groupby(group)])
  dfs.append(balanced_df)

bdf = pd.concat(dfs)

# Print resulting dataset info
print(f"Balanced dataset size: {len(bdf)}")
bdf

Balanced dataset size: 2800


,text,Agriculture,Air transportation,Battery,Bioenergy,CCUS,Coal,Combined heat and power (CHP),Defense,Electric grid,...,Rail transportation,Road transportation,Shipping,Solar,Storage,Waste,Water,Wind,one_hot_labels,idx
13219,EDP Renewables has reached an agreement to acq...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",13219
21993,the Grid uses GE’s leading Silicon Carbide (Si...,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",21993
29073,Eight GE9X test engines and two test spares ha...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",29073
11035,"de Poorter, General ManagerDEME Offshore Renew...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",11035
42162,The softball program’s mantra “Mighty Are the ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",42162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49261,"after Vindeby, Ørsted installed its 500th turb...",0,0,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,1,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",49261
1152,"The wind farm will have 23 wind turbines, each...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1152
54528,Vestas has signed a preferred supplier agreeme...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",54528
2823,AEI was created in 2014 to house the majority ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2823


In [76]:
print(bdf.shape)
balanced_ds = Dataset.from_pandas(bdf)


(2800, 31)


In [77]:
from sklearn.model_selection import train_test_split

# Split dataset into training and evaluation datasets
train_dict, eval_dict = train_test_split(balanced_ds, test_size=0.2)

# Convert dictionaries back to Dataset objects
train_dataset = Dataset.from_dict(train_dict)
eval_dataset = Dataset.from_dict(eval_dict)

# Simulate the few-shot regime by sampling 8 examples per class# 
# train_dataset = sample_dataset(train_dataset, label_column="one_hot_labels", num_samples=50)
print(train_dataset)
print(eval_dataset)


Dataset({
    features: ['text', 'Agriculture', 'Air transportation', 'Battery', 'Bioenergy', 'CCUS', 'Coal', 'Combined heat and power (CHP)', 'Defense', 'Electric grid', 'Electric vehicles', 'Gas', 'Geothermal', 'Heat pump', 'Heating', 'Hydrogen', 'Hydropower', 'Materials', 'Nuclear', 'Oil', 'Pipeline', 'Rail transportation', 'Road transportation', 'Shipping', 'Solar', 'Storage', 'Waste', 'Water', 'Wind', 'one_hot_labels', 'idx', '__index_level_0__'],
    num_rows: 2240
})
Dataset({
    features: ['text', 'Agriculture', 'Air transportation', 'Battery', 'Bioenergy', 'CCUS', 'Coal', 'Combined heat and power (CHP)', 'Defense', 'Electric grid', 'Electric vehicles', 'Gas', 'Geothermal', 'Heat pump', 'Heating', 'Hydrogen', 'Hydropower', 'Materials', 'Nuclear', 'Oil', 'Pipeline', 'Rail transportation', 'Road transportation', 'Shipping', 'Solar', 'Storage', 'Waste', 'Water', 'Wind', 'one_hot_labels', 'idx', '__index_level_0__'],
    num_rows: 560
})


In [ ]:
# Load a SetFit model from Hub
print("Downloading pretrained model...")
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2", multi_target_strategy="one-vs-rest")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    # metric="accuracy",
    metric=precision_recall_fscore_support
    batch_size=32,
    num_iterations=10, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"text": "text", "one_hot_labels": "label"}, # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
print("Training model...")
trainer.train()
metrics = trainer.evaluate()
print("Current metrics:", metrics)

# Push model to the Hub
# Format date and time as string
date_str = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M")
trainer.model.save_pretrained(f"setfit-multilabel_{date_str}_{metrics['accuracy']}")




model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Training model...


Generating Training Pairs:   0%|          | 0/10 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 58550
  Num epochs = 1
  Total optimization steps = 1830
  Total train batch size = 32


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1830 [00:00<?, ?it/s]

In [69]:
# Download from Hub and run inference
print("Getting saved model...")
# model = SetFitModel.from_pretrained("setfit-multilabel_")
# Run inference
preds = model(["ENEL wins wind plant  solar cells pv is important too", "Jinko Solar to install solar panels for Las Vegas solar farm"])

Getting saved model...


In [70]:
metrics

{'accuracy': 0.865}

In [71]:
preds

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0]])

In [48]:
# Get index of maximum value in each row
pred_idxs = [np.where(arr == 1)[0].tolist() for arr in preds]

# Get corresponding label for each index
text_labels = []
for idxs in pred_idxs:
  text_labels.append([label_cols[idx] for idx in idxs])

# Print text labels
print(text_labels)

[['Solar'], ['Solar']]
